#  RAG 10k Query

![](../images/rag-1.svg)


## Basic Setup

In [1]:
## Check if GPU is enabled
import os
import torch

## To disable GPU and experiment, uncomment the following line
## Normally, you would want to use GPU, if one is available.
# os.environ["CUDA_VISIBLE_DEVICES"]=""

print ("using CUDA/GPU: ", torch.cuda.is_available())

for i in range(torch.cuda.device_count()):
   print("device ", i , torch.cuda.get_device_properties(i).name)

using CUDA/GPU:  True
device  0 NVIDIA GeForce RTX 2070


In [2]:
## Setup logging.  To see more loging set the level to DEBUG

import sys
import logging

# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.basicConfig(stream=sys.stdout, level=logging.WARNING)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
import os, sys

this_dir = os.path.abspath('')
parent_dir = os.path.dirname(this_dir)
sys.path.append (os.path.abspath (parent_dir))

## Step-1: Load Settings

In [4]:
import os,sys
## Load Settings from .env file
from dotenv import find_dotenv, dotenv_values

# _ = load_dotenv(find_dotenv()) # read local .env file
config = dotenv_values(find_dotenv())

# debug
# print (config)

ATLAS_URI = config.get('ATLAS_URI')

if not ATLAS_URI:
    raise Exception ("'ATLAS_URI' is not set.  Please set it above to continue...")

## Only need this if we are using OpenAI for Embeddings
OPENAI_API_KEY = config.get("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise Exception ("'OPENAI_API_KEY' is not set.  Please set it above to continue...")

In [5]:
DB_NAME = 'rag1'
COLLECTION_NAME = '10k'
INDEX_NAME = 'idx_embedding'

In [6]:
import os
## LlamaIndex will download embeddings models as needed.
## Set llamaindex cache dir to ./cache dir here (Default is system tmp)
## This way, we can easily see downloaded artifacts
os.environ['LLAMA_INDEX_CACHE_DIR'] = os.path.join(os.path.abspath(''), '..', 'llama-index-cache')

In [7]:
from pymongo import MongoClient

mongodb_client = MongoClient(ATLAS_URI)

print ("Atlas client initialized")

Atlas client initialized


## Step-2 : Setup Embeddings

The default embedding is OpenAI.  We can always plugin custom embeddings

### 2.1 : OpenAI Embeddings

This is using OpenAI embedding model
You will need an API key (defined in env variable : OPENAI_API_KEY)

In [8]:
# from llama_index import  OpenAIEmbedding
# embed_model = OpenAIEmbedding()

### 2.2 : Using Custom Embeddings

Remember this embedding model must be the same as in `populate` step. 

In [9]:
from llama_index.embeddings import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

/home/sujee/anaconda3/envs/atlas-2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from llama_index import  ServiceContext
from llama_index.llms import OpenAI

# The LLM used to generate natural language responses to queries.
# If not provided, defaults to gpt-3.5-turbo from OpenAI
# If your OpenAI key is not set, defaults to llama2-chat-13B from Llama.cpp

## Here are the models available : https://platform.openai.com/docs/models
##  gpt-3.5-turbo  |   gpt-4  | gpt-4-turbo

#  | model         | context window |
#  |---------------|----------------|
#  | gpt-3.5-turbo | 4,096          |
#  | gpt-4         | 8,192          |
#  | gpt-4-turbo   | 128,000        |


## set temperature=0 for predictable results

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
## setup embed model

service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

## Step-3: Connect Illama-Index and MongoDB Atlas

Let's define MongoDB Atlas as our vector storage. This is critical to stored indexed data and then query

In [11]:
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_index.storage.storage_context import StorageContext
from llama_index.indices.vector_store.base import VectorStoreIndex


vector_store = MongoDBAtlasVectorSearch(mongodb_client = mongodb_client,
                                 db_name = DB_NAME, collection_name = COLLECTION_NAME,
                                 index_name  = 'idx_embedding',
                                 ## the following columns are set to default values
                                 # embedding_key = 'embedding', text_key = 'text', metadata_= 'metadata',
                                 )
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_vector_store(vector_store=vector_store, service_context=service_context)

## Step-4: Query Data / Ask Questions

Now that we have every thing setup, let's ask some questions

In [12]:
%%time

from IPython.display import Markdown
from pprint import pprint

response = index.as_query_engine().query("What was Uber's revenue?")
print (response)
print()
pprint(response, indent=4)

Uber's revenue for the years ended December 31, 2019, 2020, and 2021 was $13,000 million, $11,139 million, and $17,455 million, respectively.

Response(response="Uber's revenue for the years ended December 31, 2019, 2020, "
                  'and 2021 was $13,000 million, $11,139 million, and $17,455 '
                  'million, respectively.',
         source_nodes=[   NodeWithScore(node=TextNode(id_='97e43c19-36ff-4295-a7e1-cee0e271566e', embedding=None, metadata={'page_label': '54', 'file_name': 'uber_2021.pdf', 'file_path': '../data/10k/uber_2021.pdf', 'file_type': 'application/pdf', 'file_size': 1880483, 'creation_date': '2024-01-23', 'last_modified_date': '2024-01-23', 'last_accessed_date': '2024-01-29'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<No

In [13]:
%%time

from IPython.display import Markdown
from pprint import pprint

response = index.as_query_engine().query("What was Uber's revenue?  Provide answer in JSON format")
print (response)
print()
pprint(response, indent=4)

{"revenue": "$17,455 million"}

Response(response='{"revenue": "$17,455 million"}',
         source_nodes=[   NodeWithScore(node=TextNode(id_='8680dcb4-39d4-400b-ac35-32659c549d6e', embedding=None, metadata={'page_label': '98', 'file_name': 'uber_2021.pdf', 'file_path': '../data/10k/uber_2021.pdf', 'file_type': 'application/pdf', 'file_size': 1880483, 'creation_date': '2024-01-23', 'last_modified_date': '2024-01-23', 'last_accessed_date': '2024-01-29'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='d7fbeb9c-89c0-484d-a524-9c5547a0662c', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '98', 'file_name': 'uber_2021.pdf', 'file_path': '../data/10k/uber_2021.pdf', 'file_type': 'application/pdf',

In [14]:
%%time

response = index.as_query_engine().query("How much money did Lyft make in 2020?")
print (response)
print()
pprint(response, indent=4)

Lyft made $2,364,681 in revenue in 2020.

Response(response='Lyft made $2,364,681 in revenue in 2020.',
         source_nodes=[   NodeWithScore(node=TextNode(id_='e425582e-bf00-4ef2-a52f-8b83959a204c', embedding=None, metadata={'page_label': '58', 'file_name': 'lyft_2021.pdf', 'file_path': '../data/10k/lyft_2021.pdf', 'file_type': 'application/pdf', 'file_size': 1440303, 'creation_date': '2024-01-23', 'last_modified_date': '2024-01-23', 'last_accessed_date': '2024-01-29'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='9fd172b3-026a-494e-be7d-e85fe0cb480a', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '58', 'file_name': 'lyft_2021.pdf', 'file_path': '../data/10k/lyft_2021.pdf', 'file_type'

In [15]:
%%time

## The answer to this question doesn't exist in the Lyft_10k filing!
## Let's see what we get back
response = index.as_query_engine().query("How much money did Lyft make in 2018?")
print (response)
print()
pprint(response, indent=4)

I'm sorry, but I cannot answer that question based on the given context information.

Response(response="I'm sorry, but I cannot answer that question based on the "
                  'given context information.',
         source_nodes=[   NodeWithScore(node=TextNode(id_='b865ebda-59d0-49b2-b9ea-427745b89382', embedding=None, metadata={'page_label': '79', 'file_name': 'lyft_2021.pdf', 'file_path': '../data/10k/lyft_2021.pdf', 'file_type': 'application/pdf', 'file_size': 1440303, 'creation_date': '2024-01-23', 'last_modified_date': '2024-01-23', 'last_accessed_date': '2024-01-29'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='8f2ed54c-68a3-41a0-94c6-8ca8dd842553', node_type=<ObjectType.DOCUMENT: '4'>, meta

In [16]:
%%time

response = index.as_query_engine().query("When did Uber go IPO?")
print (response)
print()
pprint(response, indent=4)

Uber went IPO in 2019.

Response(response='Uber went IPO in 2019.',
         source_nodes=[   NodeWithScore(node=TextNode(id_='5a9c1dd0-44f6-4009-8427-df7e92d1ee0f', embedding=None, metadata={'page_label': '82', 'file_name': 'uber_2021.pdf', 'file_path': '../data/10k/uber_2021.pdf', 'file_type': 'application/pdf', 'file_size': 1880483, 'creation_date': '2024-01-23', 'last_modified_date': '2024-01-23', 'last_accessed_date': '2024-01-29'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='63aa7020-6388-4789-93f5-d251022b3792', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '82', 'file_name': 'uber_2021.pdf', 'file_path': '../data/10k/uber_2021.pdf', 'file_type': 'application/pdf', 'file_size': 18

### We can even query data in tables!

Here is a table based data in [Lyft's 10k document](data/10k/lyft_2021.pdf) - page 82

And the LLMs are able to extract the info and answer!

![](../images/rag-3-pdf-table.png)

In [17]:
%%time

response = index.as_query_engine().query("What were the Stock-based compensation for Lyft?")
print (response)
print()
pprint(response, indent=4)

The Stock-based compensation for Lyft was $721,710 in 2020 and $565,807 in 2019.

Response(response='The Stock-based compensation for Lyft was $721,710 in 2020 '
                  'and $565,807 in 2019.',
         source_nodes=[   NodeWithScore(node=TextNode(id_='15401925-1cde-442c-a6dd-dbe35853d28c', embedding=None, metadata={'page_label': '82', 'file_name': 'lyft_2021.pdf', 'file_path': '../data/10k/lyft_2021.pdf', 'file_type': 'application/pdf', 'file_size': 1440303, 'creation_date': '2024-01-23', 'last_modified_date': '2024-01-23', 'last_accessed_date': '2024-01-29'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='645f8284-0fdf-437c-bb47-7e3cbddf3cbc', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'p